In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

In [2]:
from vulcanai.utils import get_one_hot

from vulcanai import mnist_loader

Using cuDNN version 7103 on context None
Mapped name None to device cuda: GeForce GTX 960M (0000:01:00.0)
C:\Anaconda\Anaconda2\lib\site-packages\matplotlib\__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
(train_images, train_labels, test_images, test_labels) = mnist_loader.load_fashion_mnist()

data folder already exists
Loading training images...
Loading training labels...
Loading testing images...
Loading testing labels...


In [4]:
train_labels = get_one_hot(train_labels)

In [5]:
train_x, train_y = train_images[:40000], train_labels[:40000]
val_x, val_y = train_images[50000:60000], train_labels[50000:60000]
print("Size of:")
print("\t Training-set:\t\t{} {}".format(len(train_x), len(train_y)))
print("\t Validation-set:\t{} {}".format(len(val_x), len(val_y)))

Size of:
	 Training-set:		40000 40000
	 Validation-set:	10000 10000


In [6]:
train_class = np.argmax(train_y, axis=1)
val_class = np.argmax(val_y, axis=1)

In [7]:
train_class[:10], val_class[:10]

(array([9, 0, 0, 3, 0, 2, 7, 2, 5, 5], dtype=int64),
 array([9, 2, 1, 0, 2, 7, 9, 3, 1, 1], dtype=int64))

In [8]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"x": np.array(train_x)},
                    y=np.array(train_class),
                    num_epochs=None,
                    shuffle=True)

In [9]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"x": np.array(val_x)},
                    y=np.array(val_class),
                    num_epochs=1,
                    shuffle=False)

In [10]:
pred_x, pred_y = train_images[40100:40110], train_labels[40100:40110]
pred_class = np.argmax(pred_y, axis=1)

In [11]:
pred_class[:10]

array([9, 9, 1, 8, 1, 0, 0, 1, 4, 6], dtype=int64)

In [12]:
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": pred_x},
    num_epochs=1,
    shuffle=False)

In [13]:
train_input_fn(), val_input_fn(), pred_input_fn()

(({'x': <tf.Tensor 'random_shuffle_queue_DequeueMany:1' shape=(128, 784) dtype=float32>},
  <tf.Tensor 'random_shuffle_queue_DequeueMany:2' shape=(128,) dtype=int64>),
 ({'x': <tf.Tensor 'fifo_queue_DequeueUpTo:1' shape=(?, 784) dtype=float32>},
  <tf.Tensor 'fifo_queue_DequeueUpTo:2' shape=(?,) dtype=int64>),
 {'x': <tf.Tensor 'fifo_queue_DequeueUpTo_1:1' shape=(?, 784) dtype=float32>})

In [14]:
feature_x = tf.feature_column.numeric_column("x", shape=(28, 28))

In [15]:
feature_columns = [feature_x]

In [ ]:
feature_columns

In [16]:
num_hidden_units = [512, 256, 128]

In [17]:
model = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                   hidden_units=num_hidden_units,
                                   activation_fn=tf.nn.relu,
                                   n_classes=10,
                                   model_dir="./checkpoints_tutorial17-1/")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000007BC84E0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './checkpoints_tutorial17-1/', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


In [18]:
model.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./checkpoints_tutorial17-1/model.ckpt.
INFO:tensorflow:loss = 295.054, step = 0
INFO:tensorflow:global_step/sec: 211.416
INFO:tensorflow:loss = 66.5425, step = 100 (0.475 sec)
INFO:tensorflow:global_step/sec: 298.507
INFO:tensorflow:loss = 85.0638, step = 200 (0.335 sec)
INFO:tensorflow:global_step/sec: 302.115
INFO:tensorflow:loss = 56.6463, step = 300 (0.330 sec)
INFO:tensorflow:global_step/sec: 308.642
INFO:tensorflow:loss = 51.2977, step = 400 (0.325 sec)
INFO:tensorflow:global_step/sec: 297.619
INFO:tensorflow:loss = 50.4936, step = 500 (0.335 sec)
INFO:tensorflow:global_step/sec: 305.81
INFO:tensorflow:loss = 35.0038, step = 600 (0.328 sec)
INFO:tensorflow:global_step/sec: 288.185
INFO:tensorflow:loss 

In [19]:
result = model.evaluate(input_fn=val_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-16-05:50:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints_tutorial17-1/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-16-05:50:43
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.8747, average_loss = 0.353375, global_step = 2000, loss = 44.731


In [20]:
result

{'accuracy': 0.87470007,
 'average_loss': 0.35337502,
 'global_step': 2000,
 'loss': 44.731014}

In [21]:
predictions = model.predict(input_fn=pred_input_fn)

In [22]:
cls = [p['classes'] for p in predictions]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints_tutorial17-1/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [23]:
cls_pred = np.array(cls, dtype='int').squeeze()
cls_pred

array([9, 9, 1, 8, 1, 0, 0, 1, 4, 6])